#**Jesus Romero Alfaro**

##Tarea 9

---

Transcriba los ejercicios en una notebook de Jupyter independiente. 

Use los formatos correspondientes para escribir fórmulas. 

Responda con todos los detalles pertinentes.

Convierta los códigos vistos en clase (para calcular W, L y P(N=k)) en funciones y úselos para responder todo lo que sea posible mediante simulaciones. 

Luego verifique sus resultados utilizando las fórmulas de la teoría.

Si no es posible usar la simulación para responder alguna pregunta en particular, argumente porqué y respóndala teóricamente.

In [4]:
from random import expovariate
from statistics import mean 
from math import inf as Infinity

def mm1(lamda,mu):
#lamda = 1.3                         # Razón de arrivos (1/Lambda)  Ojo
#mu = 2.0                            # Razón de salida (1/Mu)       Ojo
  Num_Pkts = 100000                   # Numero de paquetes a simular
#T = 100000
  count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema

  tll = expovariate(lamda)            # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida
  t_evento_anterior = 0.0             # tiempo del último evento ****************Nuevas líneas de código***********

  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  r_Data = []                         # Lista de retrasos individuales  
  r_Data2 = []                         # Lista de retrasos individuales
  Area = []                           # Area de escalón
  Area2 = []                           # Area de escalón
  #Resultados=[]

  Data = {}                           # Diccionario

  while count < Num_Pkts:
  #while t < T:
    if tll < ts:                           # Evento de llegada
        t = tll

        # longitud del intervalo entre llegadas *********************************************************************
        delta = t - t_evento_anterior
        if N in Data: 
            Data[N] += delta
        else:
            Data[N] = delta

        tll_Data.append(t)
        # Area del escalón ***********************************************************************************
        if N>=1:
          Area2.append((t - t_evento_anterior)*(N-1))
        Area.append((t - t_evento_anterior)*N)
        t_evento_anterior = t

        N = N + 1.0
        tll = t + expovariate(lamda) 
        if N == 1:
            ts = t + expovariate(mu)
    else:                                         # Evento de salida
        t = ts 

        # Longitud del intervalo de tiempo **************************************************************************
        delta = t - t_evento_anterior
        if N in Data:
            Data[N] += delta
        else:
            Data[N] = delta

        ts_Data.append(t)

        # Area del escalón ***********************************************************************************
        if N>=1:
          Area2.append((t - t_evento_anterior)*(N-1))
        Area.append((t - t_evento_anterior)*N)
        t_evento_anterior = t

        N = N - 1.0
        count = count + 1                         # Paquetes simulados
        if N > 0:
            ts = t + expovariate(mu)
        else:
            ts = Infinity
            
  # Estimación del retraso promedio:
  for i in range(Num_Pkts):
  #for i in range(T):
    d = ts_Data[i] - tll_Data[i]
    r_Data.append(d)
  # Estimación del tiempo  promedio de espera
  for i in range(Num_Pkts):
  #for i in range(T):
    if i==0:
      j=0
    else:
      if (ts_Data[i-1] <= tll_Data[i]):
        j=0    #Tiempo en linea de espera 0 cuando servidor no esta ocupado 
      else:
        j = ts_Data[i-1] - tll_Data[i] #Tiempo en linea de espera  del cliente i
    r_Data2.append(j)

  # Estimación de la probabilidad de P(N=k):
  for (key, value) in Data.items():
    Data[key] = value/t

  # Se verifica que la probabilidad total es 1.0
  suma=sum(Data.values())
  print('Suma de probabilidades =',suma )

  # Se verifican los valores de expectación
  media = 0.0

  for (key, value) in Data.items():
    media = media + key*value
    print(key, round(value, 4))
  print('E[N] = ', media)

  W=round( mean(r_Data), 4)
  print( "Retraso promedio = W = ", W )
  Wq=round( mean(r_Data2), 4)
  print( "Retraso promedio = Wq = ", Wq )

  L=round(sum(Area)/t, 4)
  Lq=round(sum(Area2)/t, 4)
  print( "E[ N(t)] = L = ",L )
  print( "Lq = ",Lq )
  
  resultados = {'Media': media , 'W':W, 'L':L, 'Lq':Lq ,'Wq':Wq,'Data':Data}
  return(resultados)

# 1. En la biblioteca

Un encargado de la biblioteca es el único en el mostrador durante todo su turno de trabajo, las llegadas al mostrador siguen una distribución de Poisson con una media de 8 por hora, cada usuario de la biblioteca es atendido de uno en uno, y el tiempo de servicio sigue una
distribución exponencial con una media de 5 minutos.



a) ¿Cuál es la probabilidad de que se forme cola?

b) ¿Cuál es la longitud media de la cola?

c) ¿Cuál es el tiempo medio que un cliente pasa en la biblioteca hasta que le han
atendido?

d) ¿Cuál es el tiempo medio que un cliente pasa en la cola esperando a que le atiendan?

e) El encargado de la biblioteca pasa su tiempo en que no hay clientes clasificando artículos
de revistas. Si puede clasificar 22 fichas por hora como media cuando trabaja continuamente,
¿cuántas fichas puede ordenar durante su trabajo?

In [5]:
P1=mm1(8,12)
print("")
print("Respuesta de simulacion")
#a
d=P1['Data']
print("a)Probabilidad de que se forme cola =",(1-d[0]))
#b
print("b)Longitud media de la cola =",P1['Lq'])
#c
print("c)Tiempo medio que un cliente pasa en biblioteca=",P1['W'])
#d
print("d)Tiempo medio que un cliente pasa en la cola=",P1['Wq'])
print("")
print("Valores calculados con formula")
wq=P1['W']-(5/60)
print("Wq=W-Ws:",wq) #Ws=5/60=.083
l=P1['W']*8
print("L=W*Lambda:",l)
lq=8*wq
print("Lq=Wq*Lambda:",lq)

Suma de probabilidades = 1.0
0 0.3326
1.0 0.2205
2.0 0.1475
3.0 0.0968
4.0 0.0655
5.0 0.0457
6.0 0.0297
7.0 0.0202
8.0 0.0141
9.0 0.0092
10.0 0.0063
11.0 0.0039
12.0 0.0027
13.0 0.0017
14.0 0.0011
15.0 0.0007
16.0 0.0005
17.0 0.0004
18.0 0.0002
19.0 0.0002
20.0 0.0001
21.0 0.0001
22.0 0.0001
23.0 0.0
24.0 0.0
25.0 0.0
26.0 0.0
27.0 0.0
28.0 0.0
E[N] =  2.0299975987010144
Retraso promedio = W =  0.2538
Retraso promedio = Wq =  0.1704
E[ N(t)] = L =  2.03
Lq =  1.3626

Respuesta de simulacion
a)Probabilidad de que se forme cola = 0.6673960248647363
b)Longitud media de la cola = 1.3626
c)Tiempo medio que un cliente pasa en biblioteca= 0.2538
d)Tiempo medio que un cliente pasa en la cola= 0.1704

Valores calculados con formula
Wq=W-Ws: 0.1704666666666667
L=W*Lambda: 2.0304
Lq=Wq*Lambda: 1.3637333333333337


# 2. Lavado de autos
Un pequeño autoservicio de lavado que solo tienen capacidad para atender de uno en uno a
los autos que llegan, tiene un aparcamiento para 10 coches, incluyendo el que está siendo
lavado. La empresa ha estimado que los coches llegan siguiendo una distribución de Poisson
con una media de 20 coches/hora, el tiempo de servicio sigue una distribución exponencial
de 12 minutos. La empresa abre durante 10 horas al día.


¿Cuál es la media de coches perdidos cada día debido a las limitaciones de espacio?

In [6]:
from random import expovariate
from statistics import mean 
from math import inf as Infinity

def Problema2(lamda,mu):
#lamda = 1.3                         # Razón de arrivos (1/Lambda)  Ojo
#mu = 2.0                            # Razón de salida (1/Mu)       Ojo
  Num_Pkts = 100000                   # Numero de paquetes a simular
  T = 10
  #count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema
  autosperdidos = 0                   # Numero de autos perdidos

  tll = expovariate(lamda)            # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida
  t_evento_anterior = 0.0             # tiempo del último evento ****************Nuevas líneas de código***********

  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  r_Data = []                         # Lista de retrasos individuales
  Area = []                           # Area de escalón

  Data = {}                           # Diccionario

  #while count < Num_Pkts:
  while t < T:
    if tll < ts:                           # Evento de llegada
      if N<10:                            #Limite de 10 autos 
        t = tll
        # longitud del intervalo entre llegadas *********************************************************************
        delta = t - t_evento_anterior
        if N in Data: 
            Data[N] += delta
        else:
            Data[N] = delta

        tll_Data.append(t)

        # Area del escalón ***********************************************************************************
        Area.append((t - t_evento_anterior)*N)
        t_evento_anterior = t

        N = N + 1.0
        tll = t + expovariate(lamda) 
        if N == 1:
            ts = t + expovariate(mu)
      else:
        ts = tll   #Salidas para desocupar carril
        autosperdidos=autosperdidos + 1
    else:                                         # Evento de salida
        t = ts 

        # Longitud del intervalo de tiempo **************************************************************************
        delta = t - t_evento_anterior
        if N in Data:
            Data[N] += delta
        else:
            Data[N] = delta

        ts_Data.append(t)

        # Area del escalón ***********************************************************************************
        Area.append((t - t_evento_anterior)*N)
        t_evento_anterior = t

        N = N - 1.0
        #count = count + 1                         # Paquetes simulados
        if N > 0:
            ts = t + expovariate(mu)
        else:
            ts = Infinity
            
  # Estimación del retraso promedio:
  #for i in range(Num_Pkts):
  for i in range(T):
    d = ts_Data[i] - tll_Data[i]
    r_Data.append(d)

  # Estimación de la probabilidad de P(N=k):
  for (key, value) in Data.items():
    Data[key] = value/t

  # Se verifica que la probabilidad total es 1.0
  suma=sum(Data.values())
  #print('Suma de probabilidades =',suma )

  # Se verifican los valores de expectación
  media = 0.0

  for (key, value) in Data.items():
    media = media + key*value
    #print(key, round(value, 4))
  #print('E[N] = ', media)

  #print( "Retraso promedio = W = ", round( mean(r_Data), 4) )
  #print( "E[ N(t)] = L = ", round(sum(Area)/t, 4))
  #print("Numero de autos perdidos", autosperdidos)
  return(autosperdidos)

In [7]:
s=0
for i in range(10000):
  s=Problema2(20,5) + s
  i = i +1
p=s/10000
print("Media de coches perdidos",p)

Media de coches perdidos 141.1381


# 3. John Macko

John Macko es alumno en la U de Ozark. Hace trabajos extraños para aumentar sus ingresos.
Las peticiones de trabajo llegan en promedio cada 5 días, pero el tiempo entre ellas es
exponencial. El tiempo para terminar un trabajo también es exponencial, con una media de 4
días.

a) ¿Cuál es la probabilidad de que le falte trabajo a John?

b) Si John cobra unos $50 por cada trabajo, ¿cuál es su ingreso mensual promedio?

c) Si al final del semestre John decide subcontratar los trabajos pendientes a $40 cada uno,
¿cuánto debe esperar pagar en promedio?

In [8]:
P3=mm1(6,7.5)#Lambda=6 trabajos/mes   Mu=7.6 trabajos/mes
print("")
print("Respuesta de simulacion")
#a
d=P3['Data']
print("a)Probabilidad de que le falte trabajo a John=",d[0])
#b
print("b)Ingreso mensual promedio =",(50*P3['Media']))
#c
print("c)Pago promedio=",(40*P3['Lq']))
print("")
print("Valores calculados con formula")
wq=P3['W']-(4/30) #Ws=4/30
print("Wq=W-Ws:",wq)
l=P3['W']*(6)
print("L=W*lambda:",l)
lq=6*wq
print("Lq=Wq*lambda:",lq)

Suma de probabilidades = 1.0
0 0.1945
1.0 0.1569
2.0 0.126
3.0 0.1026
4.0 0.0828
5.0 0.0679
6.0 0.0547
7.0 0.0437
8.0 0.0344
9.0 0.028
10.0 0.0224
11.0 0.0177
12.0 0.0139
13.0 0.0116
14.0 0.0093
15.0 0.0077
16.0 0.0055
17.0 0.0039
18.0 0.0034
19.0 0.0027
20.0 0.0024
21.0 0.0017
22.0 0.0016
23.0 0.0012
24.0 0.0011
25.0 0.0009
26.0 0.0005
27.0 0.0005
28.0 0.0003
29.0 0.0002
30.0 0.0001
31.0 0.0
32.0 0.0
E[N] =  4.035253187907575
Retraso promedio = W =  0.6666
Retraso promedio = Wq =  0.5335
E[ N(t)] = L =  4.0353
Lq =  3.2297

Respuesta de simulacion
a)Probabilidad de que le falte trabajo a John= 0.19448764423465745
b)Ingreso mensual promedio = 201.76265939537873
c)Pago promedio= 129.188

Valores calculados con formula
Wq=W-Ws: 0.5332666666666667
L=W*lambda: 3.9996
Lq=Wq*lambda: 3.1996
